By default, the results of most calculations in yt are expressed in a "centimeters-grams-seconds" (CGS) set of units. This includes the values of derived fields and aliased fields.

However, this system of units may not be the most natural for a given dataset or an entire class of datasets. For this reason, yt provides the ability to define new unit systems and use them in a way that is highly configurable by the end-user. 

### Unit Systems Available in yt

Several unit systems are already supplied for use within yt. They are:

* `"cgs"`: Centimeters-grams-seconds unit system, with base of `(cm, g, s, K, radian)`. Uses the Gaussian normalization for electromagnetic units. 
* `"mks"`: Meters-kilograms-seconds unit system, with base of `(m, kg, s, K, radian, A)`.
* `"imperial"`: Imperial unit system, with base of `(mile, lbm, s, R, radian)`.
* `"galactic"`: "Galactic" unit system, with base of `(kpc, Msun, Myr, K, radian)`.
* `"solar"`: "Solar" unit system, with base of `(AU, Mearth, yr, K, radian)`. 
* `"planck"`: Planck natural units $(\hbar = c = G = k_B = 1)$, with base of `(l_pl, m_pl, t_pl, T_pl, radian)`. 
* `"geometrized"`: Geometrized natural units $(c = G = 1)$, with base of `(l_geom, m_geom, t_geom, K, radian)`. 

We can examine these unit systems by querying them from the `unit_system_registry`. For example, we can look at the default CGS system:

In [ ]:
import yt
yt.unit_system_registry["cgs"]

We can see that we have two sets of units that this system defines: "base" and "other" units. The "base" units are the set of units from which all other units in the system are composed of, such as centimeters, grams, and seconds. The "other" units are compound units which fields with specific dimensionalities are converted to, such as ergs, dynes, gauss, and electrostatic units (esu). 

We see a similar setup for the MKS system, except that in this case, there is a base unit of current, the Ampere:

In [ ]:
yt.unit_system_registry["mks"]

We can also look at the imperial system:

In [ ]:
yt.unit_system_registry["imperial"]

and the "galactic" system as well:

In [ ]:
yt.unit_system_registry["galactic"]

### Converting `YTArrays` to the Different Unit Systems

### Choosing a Unit System When Loading a Dataset

When a dataset is `load`ed, a unit system may be specified. When this happens, all aliased and derived fields will be converted to the units of the given system. The default is `"cgs"`.

For example, we can specify that the fields from a FLASH dataset can be expressed in MKS units:

In [ ]:
ds_flash = yt.load("GasSloshing/sloshing_nomag2_hdf5_plt_cnt_0100", unit_system="mks")
sp = ds_flash.sphere("c", (100.,"kpc"))
print (sp["density"]) # This is an alias for ("flash","dens")
print (sp["pressure"]) # This is an alias for ("flash","pres")
print (sp["angular_momentum_x"]) # This is a derived field
print (sp["kinetic_energy"]) # This is also a derived field

Aliased fields are converted to the requested unit system, but the on-disk fields that they correspond to remain in their original (code) units:

In [ ]:
print (sp["flash","dens"]) # This is aliased to ("gas", "density")
print (sp["flash","pres"]) # This is aliased to ("gas", "pressure")

We can take an `Enzo` dataset and express it in `"galactic"` units:

In [ ]:
ds_enzo = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030", unit_system="galactic")
sp = ds_enzo.sphere("c", (20.,"kpc"))
print (sp["density"])
print (sp["pressure"])

We can also express all of the fields associated with a dataset in that dataset's system of "code" units. Though the on-disk fields are already in these units, this means that we can express even derived fields in code units as well:

In [ ]:
ds_chombo = yt.load("KelvinHelmholtz/data.0004.hdf5", unit_system="code")
dd = ds_chombo.all_data()
print (dd["density"])
print (dd["kinetic_energy"])

### Defining Fields So That They Can Use the Different Unit Systems

If you define a new derived field for use in yt and wish to make the different unit systems available to it, you will need to specify this when calling `add_field`. Suppose I defined a new field called `"momentum_x"` and wanted it to have general units. I would have to set it up in this fashion, using the `unit_system` attribute of the dataset and querying it for the appropriate dimensions:

In [ ]:
mom_units = ds_flash.unit_system["velocity"]*ds_flash.unit_system["density"]
def _momentum_x(field, data):
    return data["density"]*data["velocity_x"]
ds_flash.add_field(("gas","momentum_x"), function=_momentum_x, units=mom_units)

Now, the field will automatically be expressed in whatever units the dataset was called with. In this case, it was MKS:

In [ ]:
slc = yt.SlicePlot(ds_flash, "z", ["momentum_x"], width=(300.,"kpc"))
slc.show()

Note that the momentum density has been plotted with the correct MKS units of $\mathrm{kg/(m^2\cdot{s})}$.

If you don't create a derived field from a dataset but instead use `yt.add_field`, and still want to use the unit system of that dataset for the units, the only option at present is to set `units="auto"` in the call to `yt.add_field` and the `dimensions` keyword to the correct dimensions for the field:

In [ ]:
from yt.units import clight

def _rest_energy(field, data):
    return data["cell_mass"]*clight*clight
yt.add_field(("gas","rest_energy"), function=_rest_energy, units="auto", dimensions="energy")

ds_flash2 = yt.load("GasSloshing/sloshing_nomag2_hdf5_plt_cnt_0150", unit_system="galactic")

sp = ds_flash2.sphere("c", (100.,"kpc"))
sp["rest_energy"]

### Obtaining Physical Constants in a Specific Unit System

Each unit system provides the ability to obtain any physical constant in yt's physical constants database in the base units of that system via the `constants` attribute of the unit system. For example, to obtain the value of Newton's universal constant of gravitation in different base units:

In [ ]:
for name in ["cgs", "mks", "imperial", "planck", "geometrized"]:
    unit_system = yt.unit_system_registry[name]
    print (name, unit_system.constants.G)

Equivalently, one could import a physical constant from the main database and convert it using `in_base`:

In [ ]:
from yt.units import G
print (G.in_base("mks"))

### Defining Your Own Unit System

You are not limited to using the unit systems already defined by yt. A new unit system can be defined by creating a new `UnitSystem` instance. For example, to create a unit system where the default units are in millimeters, centigrams, and microseconds:

In [ ]:
small_unit_system = yt.UnitSystem("small", "mm", "cg", "us")

where the required arguments are a `name` for the unit system, and the `length_unit`, `mass_unit`, and `time_unit` for the unit system, which serve as the "base" units to convert everything else to. Once a unit system instance is created, it is automatically added to the `unit_system_registry` so that it may be used throughout yt:

In [ ]:
yt.unit_system_registry["small"]

Note that the base units for the dimensions of angle and temperature have been automatically set to radians and Kelvin, respectively. If desired, these can be specified using optional arguments when creating the `UnitSystem` object:

In [ ]:
wacky_unit_system = yt.UnitSystem("wacky", "mile", "kg", "day", temperature_unit="R", angle_unit="deg")
wacky_unit_system

Though it will rarely be necessary, an MKS-style system of units where a unit of current can be specified as a base unit can also be created using the `current_mks` optional argument:

In [ ]:
mksish_unit_system = yt.UnitSystem("mksish", "dm", "ug", "ks", current_mks_unit="mA")
mksish_unit_system

Initializing a `UnitSystem` object only sets up the base units. In this case, all fields will be converted to combinations of these base units based on their dimensionality. However, you may want to specify that fields of a given dimensionality use a compound unit by default instead. For example, you might prefer that in the `"small"` unit system that pressures be represented in microdynes per millimeter squared. To do this, set these to be the units of the `"pressure"` dimension explicitly:

In [ ]:
small_unit_system["pressure"] = "udyne/mm**2"

We can now look at the `small_unit_system` object and see that these units are now defined for pressure in the "Other Units" category:

In [ ]:
small_unit_system

We can do the same for a few other dimensionalities:

In [ ]:
small_unit_system["magnetic_field_cgs"] = "mG"
small_unit_system["specific_energy"] = "cerg/ug"
small_unit_system["velocity"] = "cm/s"
small_unit_system